## Sentinel-1 data download using sentinelsat

In [2]:
# import required packages
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

import os

Definition of the environment variables and settings for downloads 

In [3]:
## change directory for download
os.chdir('D:\eo_data')

## Define search parameters

# get geoJSON file with region extent
pathJSONFile="//dapadfs/workspace_cluster_6/TRANSVERSAL_PROJECTS/MADR/COMPONENTE_2/Imagenes_Satelitales/Sentinel_2/JSON_Ibague/IbagueJSON.geojson"

# set search dates
start_date = '20170401'
end_date = '20170405'

## Specify ESA scihub credentials

# store in variables
scihub_user = 'asalazarr'
scihub_pass = 'tila8sude'


Making the call to the API

In [3]:
# connect to the API
api = SentinelAPI(scihub_user, scihub_pass, 'https://scihub.copernicus.eu/dhus')

# download single scene by known product id
# api.download(<product_id>)

# search by polygon, time, and Hub query keywords
## TO-DO: add type of product to the search terms
footprint = geojson_to_wkt(read_geojson(pathJSONFile))
products = api.query(footprint,
                     date = (start_date, end_date),
                     platformname = 'Sentinel-1')

# download all results from the search
api.download_all(products)

In [4]:
import os

# Get the metadata of the downloaded files
#own_files = api.download_all(products)[0]

# change the working directory to the location of files
os.chdir('D:\eo_data\Ibague')
print(os.getcwd())
# store the files list to a variable
eo_files = os.listdir(os.getcwd())
#print(eo_files)

D:\eo_data\Ibague


In [5]:
type(eo_files)#[0][:-3]

list

In [6]:
# Unzip files
import zipfile

# Check if a data folder exist
if not os.path.exists('data'):
    os.makedirs('data')
    print 'data folder' + ' was created'
# Check if the name of the data folder is in
if 'data' in eo_files:
    eo_files.remove('data')

##Todo catch error when a directory is in eo_files
for im_id in eo_files:    
    if not os.path.exists('data/'+im_id[:-3]+'SAFE'):
        print('Unzipping ' + im_id)
        zip_ref = zipfile.ZipFile(im_id, 'r')
        zip_ref.extractall('data')
        zip_ref.close()
    else:
        print(im_id[:-4] + ' was already uncompressed')

S1B_IW_GRDH_1SDV_20170602T104209_20170602T104234_005870_00A4B1_774C was already uncompressed
S1B_IW_GRDH_1SDV_20170614T104210_20170614T104235_006045_00A9D7_4B41 was already uncompressed
S1B_IW_GRDH_1SDV_20170626T104210_20170626T104235_006220_00AEED_0591 was already uncompressed
S1B_IW_GRDH_1SDV_20170708T104211_20170708T104236_006395_00B3E1_4D33 was already uncompressed
S1B_IW_GRDH_1SDV_20170720T104212_20170720T104237_006570_00B8E0_29C0 was already uncompressed
S1B_IW_GRDH_1SDV_20170801T104212_20170801T104237_006745_00BDE6_F318 was already uncompressed
S1B_IW_GRDH_1SDV_20170813T104213_20170813T104238_006920_00C2FF_99FD was already uncompressed
S1B_IW_GRDH_1SDV_20170825T104213_20170825T104238_007095_00C80F_6556 was already uncompressed
S1B_IW_GRDH_1SDV_20170906T104214_20170906T104239_007270_00CD25_4F8F was already uncompressed
S1B_IW_GRDH_1SDV_20170918T104214_20170918T104239_007445_00D249_A0FC was already uncompressed
S1B_IW_GRDH_1SDV_20170930T104215_20170930T104240_007620_00D74F_27DD wa

## Pre-processing using SNAP
The pre-processing workflow (to-be revised) is performed using SNAP Python API, snappy, and currently incudes the following steps:
1. Apply orbit
2. Speckle filtering
3. Terrain correction
4. Subset the area of interest
5. Logaritmic transformation (to dB)
6. Texture analysis

***

In [12]:
import snappy

from sentinelsat.sentinel import read_geojson, geojson_to_wkt
from snappy import ProductIO
from snappy import HashMap
import shutil
import os  
import ast

from snappy import GPF

GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
HashMap = snappy.jpy.get_type('java.util.HashMap')

In [17]:
## Function to get help of the SNAP operators

def Op_help(op):
        op_spi = snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(op)
        print('Op name: {}'.format(op_spi.getOperatorDescriptor().getName()))
        print('Op alias: {}\n'.format(op_spi.getOperatorDescriptor().getAlias()))
        print('PARAMETERS:\n')
        param_Desc = op_spi.getOperatorDescriptor().getParameterDescriptors()
        for param in param_Desc:
            value_set = param_Desc[0].getValueSet()
            if len(value_set) == 0:
                print('{}: {}\nDefault Value: {}\n'.format(param.getName(),param.getDescription(),param.getDefaultValue()))
            else:
                print('{}: {}\nDefault Value: {}\nPossible param: {}\n'.format(param.getName(),param.getDescription(),param.getDefaultValue(),list(value_set)))

#Op_help("Multi-Temporal-Speckle-Filter")
Op_help("CreateStack")

Op name: org.esa.s1tbx.insar.gpf.coregistration.CreateStackOp
Op alias: CreateStack

PARAMETERS:

masterBandNames: The list of source bands.
Default Value: None

slaveBandNames: The list of source bands.
Default Value: None

resamplingType: The method to be used when resampling the slave grid onto the master grid.
Default Value: NONE

extent: The output image extents.
Default Value: Master

initialOffsetMethod: Method to be used in computation of initial offset between master and slave
Default Value: Orbit



In [18]:
## Get a list of all Sentinel-1 toolbox operators
op_spi = snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpis().toString()
op_list = op_spi.split(', ')
listname = []
for op_str in op_list:
    to_add = op_str
    if op_str[0] == '[': to_add = op_str[1:]
    elif op_str[-1] == ']': to_add = op_str[:-1]
    #if to_add.split('.')[2] == 's1tbx':
    listname.append(to_add.split('$')[0])
listname.sort()
#for name in listname:
#    print(name)

# Read and process S1 products

The following code reads directories in a folder containing only S1 products. Then, creates a dictionary using the products names as keys. Then, reads and stores diferent processing steps in a second-level dictionary (i.e. inside the first dictionary). 

In [65]:
import os

# Set a variable for the location of the data
eo_direc = 'D:/eo_data/Saldana/'

# Get the names of the downloaded files
eo_files = os.listdir(eo_direc)
list(eo_files)

['S1A_IW_GRDH_1SDV_20150916T231330.SAFE',
 'S1A_IW_GRDH_1SDV_20151127T231327.SAFE',
 'S1A_IW_GRDH_1SSV_20150612T231317.SAFE',
 'S1A_IW_GRDH_1SSV_20150619T104249.SAFE',
 'S1A_IW_GRDH_1SSV_20150713T104249.SAFE',
 'S1A_IW_GRDH_1SSV_20150730T231319.SAFE',
 'S1A_IW_GRDH_1SSV_20150806T104250.SAFE',
 'S1A_IW_GRDH_1SSV_20150823T231320.SAFE',
 'S1A_IW_GRDH_1SSV_20150830T104252.SAFE',
 'S1A_IW_GRDH_1SSV_20151010T231322.SAFE',
 'S1A_IW_GRDH_1SSV_20151017T104253.SAFE',
 'S1A_IW_GRDH_1SSV_20151103T231321.SAFE',
 'S1A_IW_GRDH_1SSV_20151110T104241.SAFE',
 'S1A_IW_GRDH_1SSV_20151204T104300.SAFE',
 'S1A_IW_GRDH_1SSV_20151221T231315.SAFE',
 'S1A_IW_GRDH_1SSV_20160114T231328.SAFE',
 'S1A_IW_GRDH_1SSV_20160121T104259.SAFE',
 'S1A_IW_GRDH_1SSV_20160207T231327.SAFE',
 'S1A_IW_GRDH_1SSV_20160214T104258.SAFE',
 'S1A_IW_GRDH_1SSV_20160302T231327.SAFE',
 'S1A_IW_GRDH_1SSV_20160309T104258.SAFE',
 'S1A_IW_GRDH_1SSV_20160402T104259.SAFE',
 'S1A_IW_GRDH_1SSV_20160419T231329.SAFE']

In [66]:
## Create a dictionary to read Sentinel-1 L1 GRD products
product = {}
for element in eo_files:
    product[element[:-5]] = {}

# Define the area of interest
WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')
regPath = "//dapadfs/workspace_cluster_6/TRANSVERSAL_PROJECTS/MADR/COMPONENTE_2/Imagenes_Satelitales/Sentinel_2/" + \
                      "JSON_Saldana/saldanaextent.geojson"
geom = geojson_to_wkt(read_geojson(regPath))

# Define polarizations of interest
polarizations = ['VV', 'VH']

# Final results dictionary, with lists
results = []
#for pol in polarizations:
#    results[pol] = []

In [67]:
product.keys()

['S1A_IW_GRDH_1SSV_20151017T104253',
 'S1A_IW_GRDH_1SSV_20160419T231329',
 'S1A_IW_GRDH_1SSV_20150612T231317',
 'S1A_IW_GRDH_1SDV_20150916T231330',
 'S1A_IW_GRDH_1SSV_20150806T104250',
 'S1A_IW_GRDH_1SSV_20160309T104258',
 'S1A_IW_GRDH_1SSV_20151221T231315',
 'S1A_IW_GRDH_1SSV_20150823T231320',
 'S1A_IW_GRDH_1SSV_20160121T104259',
 'S1A_IW_GRDH_1SSV_20160207T231327',
 'S1A_IW_GRDH_1SSV_20160214T104258',
 'S1A_IW_GRDH_1SSV_20151204T104300',
 'S1A_IW_GRDH_1SSV_20150619T104249',
 'S1A_IW_GRDH_1SSV_20150730T231319',
 'S1A_IW_GRDH_1SSV_20151110T104241',
 'S1A_IW_GRDH_1SSV_20151010T231322',
 'S1A_IW_GRDH_1SSV_20160402T104259',
 'S1A_IW_GRDH_1SSV_20150830T104252',
 'S1A_IW_GRDH_1SSV_20151103T231321',
 'S1A_IW_GRDH_1SDV_20151127T231327',
 'S1A_IW_GRDH_1SSV_20160302T231327',
 'S1A_IW_GRDH_1SSV_20160114T231328',
 'S1A_IW_GRDH_1SSV_20150713T104249']

In [68]:
for key, value in product.iteritems():    
    # Read the product
    value['GRD'] = ProductIO.readProduct(eo_direc+key+'.SAFE/manifest.safe')
    print('Reading '+key)
    
    # Apply orbit
    param_orbit = HashMap()
    value['orbit'] = GPF.createProduct("Apply-Orbit-File", param_orbit, value['GRD'])
    
    # The following operations are specific por each polarization    
    for pol in polarizations:
        # Radiometric calibration
        param_calibration = HashMap()
        param_calibration.put('outputSigmaBand', True)
        param_calibration.put('sourceBands', getBandNames(value['orbit'], 'Intensity_'+pol))
        param_calibration.put('selectedPolarisations', pol)
        param_calibration.put('outputImageScaleInDb', False)
        value['calibration_'+pol] = GPF.createProduct("Calibration", param_calibration, value['orbit'])
        
        # Terrain correction
        param_terraincor = HashMap()
        param_terraincor.put('demResamplingMethod', 'NEAREST_NEIGHBOUR')
        param_terraincor.put('imgResamplingMethod', 'NEAREST_NEIGHBOUR')
        param_terraincor.put('applyRadiometricNormalization', True)
        param_terraincor.put('demName', 'SRTM 3Sec')
        param_terraincor.put('pixelSpacingInMeter', 10.0)
        param_terraincor.put('sourceBands', getBandNames(value['calibration_'+pol], 'Sigma0_'+pol))
        param_terraincor.put('mapProjection', 'WGS84(DD)')
        value['terraincor_'+pol] = GPF.createProduct("Terrain-Correction", param_terraincor, value['calibration_'+pol])
        
        # Subset to area of interest
        param_subset = HashMap()
        param_subset.put('geoRegion', geom)
        param_subset.put('outputImageScaleInDb', False)
        param_subset.put('sourceBandNames', getBandNames(value['terraincor_'+pol], 'Sigma0_'+pol))
        value['subset_'+pol] = GPF.createProduct("Subset", param_subset, value['terraincor_'+pol])
        
        # define the name of the output
        #output_name = eo_direc + 'prep/' + key + "_" + pol + "_"
        
        # Write the results to files
        #ProductIO.writeProduct(value['subset_'+pol], output_name, 'BEAM-DIMAP')
        results.append(value['subset_'+pol])#product
        
        # dispose all the intermediate products
        #value['calibration_'+pol].dispose()
        #value['terraincor_'+pol].dispose()
        #value['subset_'+pol].dispose()
        
    #dispose all the intermediate products
    #value['GRD'].dispose()
    #value['orbit'].dispose()


Reading S1A_IW_GRDH_1SSV_20151017T104253
Reading S1A_IW_GRDH_1SSV_20160419T231329
Reading S1A_IW_GRDH_1SSV_20150612T231317
Reading S1A_IW_GRDH_1SDV_20150916T231330
Reading S1A_IW_GRDH_1SSV_20150806T104250
Reading S1A_IW_GRDH_1SSV_20160309T104258
Reading S1A_IW_GRDH_1SSV_20151221T231315
Reading S1A_IW_GRDH_1SSV_20150823T231320
Reading S1A_IW_GRDH_1SSV_20160121T104259
Reading S1A_IW_GRDH_1SSV_20160207T231327
Reading S1A_IW_GRDH_1SSV_20160214T104258
Reading S1A_IW_GRDH_1SSV_20151204T104300
Reading S1A_IW_GRDH_1SSV_20150619T104249
Reading S1A_IW_GRDH_1SSV_20150730T231319
Reading S1A_IW_GRDH_1SSV_20151110T104241
Reading S1A_IW_GRDH_1SSV_20151010T231322
Reading S1A_IW_GRDH_1SSV_20160402T104259
Reading S1A_IW_GRDH_1SSV_20150830T104252
Reading S1A_IW_GRDH_1SSV_20151103T231321
Reading S1A_IW_GRDH_1SDV_20151127T231327
Reading S1A_IW_GRDH_1SSV_20160302T231327
Reading S1A_IW_GRDH_1SSV_20160114T231328
Reading S1A_IW_GRDH_1SSV_20150713T104249


In [69]:
stack = stacking(results)

Creating stack...


In [70]:
print('Multi-temporal speckle VV')
speckle_VV = mtspeckle_sigma0(stack, 'VV')
print(getBandNames(speckle_VV))

print('Multi-temporal speckle VH')
speckle_VH = mtspeckle_sigma0(stack, 'VH')
print(getBandNames(speckle_VH))

Multi-temporal speckle VV
Sigma0_VV_mst_14Jan2016,Sigma0_VV_slv1_02Mar2016,Sigma0_VV_slv3_13Jul2015,Sigma0_VV_slv4_10Oct2015,Sigma0_VV_slv5_30Aug2015,Sigma0_VV_slv6_02Apr2016,Sigma0_VV_slv7_27Nov2015,Sigma0_VV_slv8_03Nov2015,Sigma0_VV_slv9_09Mar2016,Sigma0_VV_slv10_23Aug2015,Sigma0_VV_slv11_21Dec2015,Sigma0_VV_slv12_14Feb2016,Sigma0_VV_slv13_19Jun2015,Sigma0_VV_slv14_04Dec2015,Sigma0_VV_slv15_10Nov2015,Sigma0_VV_slv16_30Jul2015,Sigma0_VV_slv18_06Aug2015,Sigma0_VV_slv19_16Sep2015,Sigma0_VV_slv20_12Jun2015,Sigma0_VV_slv21_21Jan2016,Sigma0_VV_slv22_07Feb2016,Sigma0_VV_slv23_19Apr2016,Sigma0_VV_slv24_17Oct2015
Multi-temporal speckle VH
Sigma0_VH_slv2_27Nov2015,Sigma0_VH_slv17_16Sep2015


In [71]:
print('Writing VH')
ProductIO.writeProduct(Sigma0_todB(speckle_VH), eo_direc + 'VH_specklefil_dB', 'BEAM-DIMAP')
print('Writing VV')
ProductIO.writeProduct(Sigma0_todB(speckle_VV), eo_direc + 'VV_specklefil_dB', 'BEAM-DIMAP')

Writing VH
Writing VV


RuntimeError: java.lang.NullPointerException

In [ ]:
for key, value in product.iteritems():
    for pol in polarizations:
        value['calibration_'+pol].dispose()
        value['terraincor_'+pol].dispose()
        value['subset_'+pol].dispose()
    value['GRD'].dispose()
    value['orbit'].dispose()
stack.dispose()
speckle_VV.dispose()
speckle_VH.dispose()

### Speckle filtering 

* Multi-temporal speckle filtering
* Scale transformation to dB

In [80]:
## Read all the VV/VH files and store in a list (for further pre-processing)
import os, re

filenames_VV = filter(re.compile(r'VV_\.dim').search, os.listdir(os.getcwd()+'\data\prep'))
filenames_VH = filter(re.compile(r'VH_\.dim').search, os.listdir(os.getcwd()+'\data\prep'))

VV_l = []
for eofile in pre_VV:
    VV_l.append(ProductIO.readProduct(os.getcwd()+'/data/prep/' + eofile))

VH_l = []
for eofile in pre_VH:
    VH_l.append(ProductIO.readProduct(os.getcwd()+'/data/prep/' + eofile))
    
## Apply multi-temporal speckle filtering
sf_product_VV = mtspeckle_sigma0(VV_stack)
sf_product_VH = mtspeckle_sigma0(VH_stack)

# Change to logaritmic units (dB) 
db_product_VV = Sigma0_todB(sf_product_VV)
db_product_VH = Sigma0_todB(sf_product_VH)

## Write the results
ProductIO.writeProduct(db_product_VV, eo_direc + 'prep/VH_specklefil_dB', 'BEAM-DIMAP')
ProductIO.writeProduct(db_product_VH, eo_direc + 'prep/VH_specklefil_dB', 'BEAM-DIMAP')

In [61]:
## Function to apply multi-temporal speckle filter of a stacked product Sigma0 bands
"""
Multi-Temporal-Speckle-Filter Op Parameters:
    filter: None --- Default Value: Lee Sigma
    nfilterSizeX: The kernel x dimension --- Default Value: 3
    nfilterSizeY: The kernel y dimension --- Default Value: 3
    ndampingFactor: The damping factor (Frost filter only) --- Default Value: 2
    nestimateENL: None --- Default Value: false
    nenl: The number of looks --- Default Value: 1.0
    nnumLooksStr: None --- nDefault Value: 1
    nwindowSize: None --- Default Value: 7x7
    ntargetWindowSizeStr: None --- Default Value: 3x3
    nsigmaStr: None --- Default Value: 0.9
    nanSize: The Adaptive Neighbourhood size --- Default Value: 50
"""

import re

def getBandNames (product, sfilter = ''):
    """
    Produces a string to use in the sourceBandNames parameter specification of SNAP operators.
    Args:
        product (): 
        sfilter (string): regular expression to filter the name of the bands
    Output:
        returns a string with comma-separated band names
    """
    band_names = product.getBandNames()
    if sfilter != '':
        band_names = filter(re.compile(r''+sfilter).search, band_names)
    if len(band_names) > 0:
        band_names = ','.join(band_names)
    else:
        band_names = None
    return band_names

def mtspeckle_sigma0 (stacked_prod, pol):
    """
    Applies the a multi-temporal speckle filter to the a corregistered calibrated product stack. Takes the product bands
    which name starts with 'Sigma0'.
    
    Args:
        stacked_prod (): product with all the bands to be used for the multi-temporal speckle filter operation
        pol (str): polarization to apply the speckle filter (VV or VH)
    Output:
    """
    param_specklefilter = HashMap()
    param_specklefilter.put('sourceBandNames', getBandNames(stacked_prod, "Sigma0_"+pol))
    param_specklefilter.put('filter', 'Lee Sigma')
    sf_product = GPF.createProduct("Multi-Temporal-Speckle-Filter", param_specklefilter, stacked_prod)
    return sf_product

def Sigma0_todB (product):
    """
    Transforms the product bands to a logaritmic scale in dB (10*log10[band]).
    
    Args:
        product: product with Sigma0 bands in linear units
    Output:
    """
    param_logdB = HashMap()
    param_logdB.put('sourceBandNames', getBandNames(product))
    db_product = GPF.createProduct("LinearToFromdB", param_logdB, product)
    return db_product

In [20]:
## Function that iterates over the list of products and stacks the its bands using the SNAP Collocate Operation

def stacking(product_set):
    # define the stack parameters
    params = HashMap()
    params.put('resamplingType', None)
    params.put('initialOffsetMethod', 'Product Geolocation')
    params.put('extent', 'Master')
    
    # create the stack
    print("Creating stack...")
    create_stack = GPF.createProduct('CreateStack', params, product_set)
    
    return create_stack


def stacking_ (prod_list):
    """
    Recursive function. Takes a list of SNAP products and returns a stacked product with all the bands named with
    the products acquisition dates.
    Args:
        prod_list: a list of products to be stacked
    Output: returns an individual product with the bands of the other products 
    """
    if len(prod_list) == 1:
        return prod_list[-1]
    else:
        param_coll = HashMap()
        input_coll = HashMap()
        input_coll.put("master", prod_list[0])
        input_coll.put("slave", prod_list[1])
        param_coll.put("masterComponentPattern","${ORIGINAL_NAME}_"+prod_list[0].getName()[17:31])
        param_coll.put("slaveComponentPattern","${ORIGINAL_NAME}_"+prod_list[1].getName()[17:31])
        prod_list[1] = GPF.createProduct('Collocate', param_coll, input_coll)
        return stacking_(prod_list[1:])


### Texture analysis

In [18]:

# paramaters for GLCM texture analysis
paramGLCM = HashMap()
paramGLCM.put('sourceBandNames', 'Sigma0_' + 'VV')
paramGLCM.put('windowSizeStr', '5x5')
paramGLCM.put('quantizerStr', 'Probabilistic Quantizer')
paramGLCM.put('quantizationLevelsStr', '16')
paramGLCM.put('displacement','4' )
paramGLCM.put('outputContrast','true')
paramGLCM.put('outputDissimilarity','true')
paramGLCM.put('outputHomogeneity','true')
paramGLCM.put('outputASM','true')
paramGLCM.put('outputEnergy','true')
paramGLCM.put('outputMean','true')
paramGLCM.put('outputVariance','true')
paramGLCM.put('outputCorrelation','true')

product = GPF.createProduct("GLCM", paramGLCM, product)

#GLCM_output = "C:/Users/asalazar/Documents/Ibague/_subset_VV"

#ProductIO.writeProduct(targetGCLM, GLCM_output, 'BEAM-DIMAP')


In [21]:
# Monitor function. Could be useful for monitoring progress of file writing. To test!

from snappy import jpy

def createProgressMonitor():
    PWPM = jpy.get_type('com.bc.ceres.core.PrintWriterProgressMonitor')
    JavaSystem = jpy.get_type('java.lang.System')
    monitor = PWPM(JavaSystem.out)
    return monitor

ProductIO.writeProduct(product, 'C:/Users/asalazar/Documents/Ibague/GLCM_output', 'BEAM-DIMAP', pm = createProgressMonitor())